In [14]:
import pandas as pd 
import sqlite3

In [15]:
connect_db = sqlite3.connect('checking-logs.sqlite')
query = 'PRAGMA table_info(test);'
pd.io.sql.read_sql(query, connect_db)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [16]:
query = 'select * from test limit 10'
pd.io.sql.read_sql(query, connect_db)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [17]:
query = '''
    select  uid, 
	        min(cast((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS Integer)) as min 
    from test 
    left join deadlines on test.labname = deadlines.labs 
    where labname <> 'project1'
    '''
df_min = pd.io.sql.read_sql(query, connect_db)
df_min

,uid,min
0,user_30,-202


In [18]:
query = '''
        select uid, 
	        max(cast((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS Integer)) as max 
        from test 
        left join deadlines on test.labname = deadlines.labs 
        where labname <> 'project1'
        '''
df_max = pd.io.sql.read_sql(query, connect_db)
df_max

,uid,max
0,user_25,-2


In [19]:
query = '''
        select uid, 
	        avg(cast((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS Integer)) as avg 
        from test 
        left join deadlines on test.labname = deadlines.labs 
        where labname <> 'project1'
        '''
df_avg = pd.io.sql.read_sql(query, connect_db)
df_avg

,uid,avg
0,user_30,-89.125


In [20]:
query = '''
select test.uid, 
		avg(cast((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS Integer)) as avg_diff, 
		views.pageviews
from test 
left join deadlines on test.labname = deadlines.labs 
left join 
		(select pageviews.uid, 
     		count(1) as pageviews 
		from pageviews 
     	group by pageviews.uid) as views on views.uid = test.uid
where labname <> 'project1' group by test.uid
'''
views_diff = pd.io.sql.read_sql(query, connect_db)
views_diff


,uid,avg_diff,pageviews
0,user_1,-64.400000,28
1,user_10,-74.800000,89
2,user_14,-159.000000,143
3,user_17,-61.600000,47
4,user_18,-5.666667,3
5,user_19,-98.750000,16
6,user_21,-95.500000,10
7,user_25,-92.600000,179
8,user_28,-86.400000,149
9,user_3,-105.400000,317


In [21]:
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


In [22]:
connect_db.close()